# Project 2: KR210 Serial Manipulator Kinematics

## IK_debug.py
- direct import from IK_server.py
    Avoids repetitive and error-prone process of manually copy-pasting into IK_debug.py
    For simplicity/operability, moved IK_debug.py to scripts folder
    required corresponding modifications to IK_server, now tested and stable.
- added test case zero, corresponding to the neutral/zero configuration of the arm, as specified by the DH parameters.
- improved output text formatting
    Added run start date/time for easier
    Added more detail, visual cuing, etc.

- added forward kinematics for joint value verification (as recommended)



## IK_server.py
### Contributions
- position error performance logging
- automatic plot generation; include plots, maybe try to get working again?
- ubiquitous instances of literalization/hard-coding for steps that did not require repetition in the main loop:
    - instances here...
    -

- ubiquitous introduction of numpy for speed
    Motivating rationale
    - instances here...
    -

Analyses performed; notebooks/scripts to perform them; illustrations

attempts to modify safe spawner script?
- did that really get me anything?
- at least mention difficulties with simulator/os performance, stability, etc.

**=============================================================================================**

**Suggestions to Make Your Project Stand Out!**

**To have a standout submission, calculate and plot the error in end-effector pose generated by your joint angle commands. You can do this by calculating end-effector poses via Forward Kinematics using your code output (which is a set of joint angles) and comparing it with the end-effector poses you received as input.**

**=============================================================================================**


**=============================================================================================**

**What to include in your submission:**

**You may submit your project as a zip file or with a link to a GitHub repo. The submission must include these items:**
**IK_server.py file with your code (make sure to add comments at appropriate places in your code)**
**Writeup report (md or pdf file)**

**=============================================================================================**



# Kinematic Analysis of KUKA KR210
## Joel Tiura
***

**=============================================================================================**

**CRITERIA: Provide a Writeup / README that includes all the rubric points and how you addressed each one. You can submit your writeup as markdown or pdf. Here is a template writeup for this project you can use as a guide and a starting point.**

**MEETS SPECIFICATIONS: The writeup / README should include a statement and supporting figures / images that explain how each rubric item was addressed, and specifically where in the code each step was handled.**

**=============================================================================================**
## Introduction / Abstract
## Forward Kinematics Functionality as Design Prerequisite
### General DH Transform Matrices, Associated Utilities



## Forward Kinematics Tool

## Analysis of Robot Geometry and Analytical Model Definition
***
### Denavit-Hartenberg Parameterization
#### URDF File Links
##### Link Lengths
##### Link Geometry
##### Link Origins

#### URDF File Joints
##### Joint Types
##### Joint Coordinates
##### Joint Ranges

#### Composing DH-Compatible Geometric Model
#### Composing DH-Compatible Geometric Model

#### DH Parameter Table
**=============================================================================================**

**CRITERIA: Run the forward_kinematics demo and evaluate the kr210.urdf.xacro file to perform kinematic analysis of Kuka KR210 robot and derive its DH parameters.**

**MEETS SPECIFICATIONS: Your writeup should contain a DH parameter table with proper notations and description about how you obtained the table. Make sure to use the modified DH parameters discussed in this lesson. Please add an annotated figure of the robot with proper link assignments and joint rotations (Example figure provided in the writeup template). It is strongly recommended that you use pen and paper to create this figure to get a better understanding of the robot kinematics.**

**=============================================================================================**


Links | alpha(i-1) | a(i-1) | d(i-1) | theta(i)
--- | --- | --- | --- | ---
0->1 | 0 | 0 | 0.75 | q1
1->2 | - pi/2 |   0.35 |      0 | q2 - pi/2
2->3 | 0 |   1.25 |      0 | q3
3->4 | - pi/2 | -0.054 |   1.50 | q4
4->5 | pi/2 |      0 |      0 | q5
5->6 | - pi/2 |      0 |      0 | q6
6->EE | 0 |      0 |  0.303 | 0


#### Generate DH FK Transform Matrices from Table

**=============================================================================================**

**CRITERIA: Using the DH parameter table you derived earlier, create individual transformation matrices about each joint. In addition, also generate a generalized homogeneous transform between base_link and gripper_link using only end-effector(gripper) pose.**

**MEETS SPECIFICATIONS: Your writeup should contain individual transform matrices about each joint using the DH table and a homogeneous transform matrix from base_link to gripper_link using only the position and orientation of the gripper_link. These matrices can be created using any software of your choice or hand written. Also include an explanation on how you created these matrices.**

**=============================================================================================**




**=============================================================================================**

**CRITERIA: Decouple Inverse Kinematics problem into Inverse Position Kinematics and inverse Orientation Kinematics; doing so derive the equations to calculate all individual joint angles.**

**MEETS SPECIFICATIONS: Based on the geometric Inverse Kinematics method described here, breakdown the IK problem into Position and Orientation problems. Derive the equations for individual joint angles. Your writeup must contain details about the steps you took to arrive at those equations. Add figures where necessary. If any given joint has multiple solutions, select the best solution and provide explanation about your choice (Hint: Observe the active robot workspace in this project and the fact that some joints have physical limits).**

**=============================================================================================**

## Module Implementation: IK_server.py
### Design Overview
In my preparatory analysis and early experiments developing a set of inverse kinematics equations, I noticed that most of the problem was completely general and invariant over all poses, and only needed to be derived once in a theoretical context. That context was using SymPy in Jupyter notebooks, where I could leverage the power of the notebooks' cell-based execution to streamline exploratory work. 

SymPy was a delight to use, and helped me make some clever optimizations. By using its robust computer algebra system to compute exact symbolic relations, I was able to derive a coordinate system correction matrix which was not to make performance improvements in code that di, and using

As I started implementing a solution, I found that I didn't like the suggested organization of the handle_calculate_IK function. It struck me as inelegant to be programmatically constructing and manipulating invariant constructs solution inside the working server code. and as such, I sought to  as many features underlying problem structure as possible. much of the process of deriving the components of a working solution was  to distill the inverse kinematics problem down to its essence, and do my best to refine the IK_server loop into a representation of a solution to that problem.
weed out redundant calculations, the minimum number of 'moving parts'

### Function: handle_calculate_IK
r_EE is the rotational component of the transformation matrix between the base coord
roll, pitch, and yaw specify end-effector orientation relative to the base coordinate system,
but only when the corresponding sequence of rotations is applied with respect to static coordinates, and
in an XYZ axis order.

This sequence and convention is not a coincidence; it's the is the default behavior of the
tf.transformations.euler_from_quaternion() function.

```
(roll, pitch, yaw) = tf.transformations.euler_from_quaternion([
    req.poses[x].orientation.x,
    req.poses[x].orientation.y,
    req.poses[x].orientation.z,
    req.poses[x].orientation.w
    ])
```
These values of roll, pitch, and yaw are calculated using the 'sxyz' convention defined in the tf library,
where *'s'* denotes static frame, and *'xyz'* gives an axis sequence for the rotations.

while the wrist center's location is not specified explicitly, it is already determined
*implicitly* by the specified end effector orientation.
     

`r_EE` is the general symbolic rotation matrix for the end effector orientation, defined outside loop.
r_EE represents this matrix for a given pose, substituting in the roll, pitch, and yaw variables
extracted from the pose orientation quaternion.


### Function: construct_R_EE
#### Derivation
#### Implementation
```
def construct_R_EE(rpy_orientation_tuple):
    '''
    construct_R_EE(orientation: Tuple[float, float, float]) -> np.ndarray:

    :param orientation: tuple containing roll, pitch, and yaw values extracted from the pose orientation quaternion
    :return: R_EE, 3x3 numpy ndarray representing the rotation matrix between the base link and the end effector

    Injects pose-specific roll/pitch/yaw euler angles into a preconfigured matrix representing the rotation transform
    between the base link coordinates and the end effector orientation in those coordinates.
    NOTE: The array construction below encodes the necessary coordinate system correction, included in its derivation.
    '''
    # unpack roll/pitch/yaw euler angles
    r, p, y = rpy_orientation_tuple
    
    # evaluate all trigonometric expressions before creating array
    cos_p = cos(p)
    sin_p = sin(p)
    cos_y = cos(y)
    sin_y = sin(y)
    cos_r = cos(r)
    sin_r = sin(r)
    
    # R_EE is a precise symbolically-derived ('pre-fab') end effector orientation matrix.
    R_EE = np.array([
        [sin_p * cos_r * cos_y + sin_r * sin_y,    -sin_p * sin_r * cos_y + sin_y * cos_r,    cos_p * cos_y],
        [sin_p * sin_y * cos_r - sin_r * cos_y,    -sin_p * sin_r * sin_y - cos_r * cos_y,    sin_y * cos_p],
        [                        cos_p * cos_r,                            -sin_r * cos_p,          -sin_p]],
        dtype=np.float64)
    
    return R_EE
```

As with most of my work on this project, I designed this function to be fast and to encode as much of the problem's
invariant features into its literal structure as possible, factoring the theoretical analysis out of the engineered
solution.

This implementation opts to eliminate the redundant, invariant procedure of instantiating a matrix object to 
represent the coordinate system correction matrix (R_corr), adding more code and computational load,

error correction transform  for the coordinate system mismatch by evaluating substitutions numerically, which leads
to small but avoidable error, I opted to solve it symbolically and just multiply it into the total end effector 
transform which is then represented as exact symbolic formulae, cleanly expressed in static code, using numpy
trig functions in place of their sympy equivalents.

Includes coordinate correction implicitly by having multiplied in the following matrix during the derivation of the complete transform:
```
R_corr = Matrix([[ 0,  0,  1],
                 [ 0, -1,  0],
                 [ 1,  0,  0]])
```
Where it has been demonstrated that given:
```
x = Matrix([[1], [0], [0]]) # == column((1, 0, 0));
y = Matrix([[0], [1], [0]]) # == column((0, 1, 0));
z = Matrix([[0], [0], [1]]) # == column((0, 0, 1));
```
The following desired equalities hold:
```
R_corr * x == Matrix([[0], [ 0], [1]]) ==  z
R_corr * y == Matrix([[0], [-1], [0]]) == -y
R_corr * y == Matrix([[1], [ 0], [0]]) ==  x
```
So it is certainly the correct transform.


### Function: compute_EE_position_error
#### Derivation
#### Implementation
```
def compute_EE_position_error(theta_list, pose):
    '''
    evaluate_EE_error(theta_list: List[float], pose: Pose) -> Tuple[float, float, float, float]:
    
    x_error, y_error, z_error, absolute_error = evaluate_EE_error([theta1, theta2, theta3, theta4, theta5, theta6])
    
    :param theta_list: calculated values for joints 1:6 in ascending order
    :param pose: pose object contained in req.poses for which the given theta list was calculated
    :return: x_err, y_err, z_err, abs_err
    
    Calculates spatial positioning error of the end effector given the theta values calculated via inverse kinematics.
    The error is calculated by comparing the position requested in 'pose' with a the vector 'fk_position', which is 
    computed by substituting the given theta values into the translation column of a total homogeneous DH transform
    matrix. 
    '''
    # unpack theta values
    q1, q2, q3, q4, q5, q6 = theta_list

    # substitute joint angle values into  between the base link and the end effector
    # fk_position as represented also has frame error correction baked in
    fk_position = np.array([
        [-0.303 * (sin(q1) * sin(q4) + sin(q2 + q3) * cos(q1) * cos(q4)) * sin(q5)
         + (1.25 * sin(q2) - 0.054 * sin(q2 + q3) + 1.5 * cos(q2 + q3) + 0.35) * cos(q1)
         + 0.303 * cos(q1) * cos(q5) * cos(q2 + q3)],
        [-0.303 * (sin(q1) * sin(q2 + q3) * cos(q4) - sin(q4) * cos(q1)) * sin(q5)
         + (1.25 * sin(q2) - 0.054 * sin(q2 + q3) + 1.5 * cos(q2 + q3) + 0.35) * sin(q1)
         + 0.303 * sin(q1) * cos(q5) * cos(q2 + q3)],
        [-0.303 * sin(q5) * cos(q4) * cos(q2 + q3) - 0.303 * sin(q2 + q3) * cos(q5)
         - 1.5 * sin(q2 + q3) + 1.25 * cos(q2) - 0.054 * cos(q2 + q3) + 0.75]], dtype=np.float64)
    
    pose_target = np.array([[pose.position.x], [pose.position.y], [pose.position.z]])
    error_vect = pose_target - fk_position
    x_err, y_err, z_err = error_vect[0][0], error_vect[1][0], error_vect[2][0]
    abs_err = sqrt(error_vect[0][0] ** 2 + error_vect[1][0] ** 2 + error_vect[2][0] ** 2)
    
    return x_err, y_err, z_err, abs_err
```
It is sufficient to evaluate only this three-element slice of the transform matrix, so the complete matrix is not 
represented in this function, but was tested and shown to be a valid model of the KR210 during the design process.



### Function: construct_R0_3_inverse
#### Derivation
#### Implementation
```
def construct_R0_3_inverse(q1, q2, q3):
    '''
    construct_R0_3_inverse(q1: float, q2: float, q3: float) -> np.ndarray:

    :param q1: theta1 numeric value
    :param q2: theta2 numeric value
    :param q3: theta3 numeric value
    :return: 3x3 numpy array (dtype = np.float64); inverse of R0_3

    Substitutes the first three joint angles into a pre-determined matrix expression which encodes
    the inverse of the rotation matrix between link_0 and link_3 and returns this as a numpy float64 array.
    '''
    R0_3_inverse = np.array([
        [ sin(q2 + q3) * cos(q1), sin(q1) * sin(q2 + q3),  cos(q2 + q3)],
        [ cos(q1) * cos(q2 + q3), sin(q1) * cos(q2 + q3), -sin(q2 + q3)],
        [               -sin(q1),                cos(q1),             0]], dtype=np.float64)
    
    return R0_3_inverse
```

This condensed, direct invocation of the rotation matrix inverse is an attempt at elegant minimalism, along
the lines of 'less code, less trouble.' Because so many properties of the system are invariant between poses,
it's possible to abstract a lot of theory out of the implementation. This array creation expression was derived
symbolically using sympy and then translated into a numpy ndarray constructor and numpy trig functions.

Part of the appeal of this implementation for me is it's further potential for vectorization. While the rest of the
code base would need adjusting before this would bear fruit, it should not be too difficult to pass in vector args,
and return an array of transform matrices calculated with even more efficiency.


#### Design Considerations and Philosophy
##### Sympy is for Theory; Numpy is for Practice
##### Baking in Invariant Quantities and Computations
##### IK_debug.py Integration
##### Position Error Calculation and Logging
##### Error Visualization - Automated Image Output with Matplotlib

#### Key Features and Selected Highlights
#### Extensions, Future Plans, Unfinished Features
- re-implement sympy matrix derivations from jupyter notebooks as automatic constructor functions;
- include functionality to pickle these so that they can be loaded faster and into other code.
- enhances reproducibility and transparency of algorithm.
**=============================================================================================**

**CRITERIA: Fill in the IK_server.py file with properly commented python code for calculating Inverse Kinematics based on previously performed Kinematic Analysis. Your code must guide the robot to successfully complete 8/10 pick and place cycles. A screenshot of the completed pick and place process is included.**

**MEETS SPECIFICATIONS: IK_server.py must contain properly commented code. The robot must track the planned trajectory and successfully complete pick and place operation. Your writeup must include explanation for the code and a discussion on the results, and a screenshot of the completed pick and place process.**

**=============================================================================================**
### IK_debug.py
#### Direct `import handle_ik_request` from IK_server.py
Avoids repetitive and error-prone process of manually copy-pasting into IK_debug.py
For simplicity/operability, moved IK_debug.py to scripts folder
This required only minimal modifications to IK_server, now tested and stable.

#### Added Test Case 0 (Neutral Pose) 
corresponding to the neutral/zero configuration of the arm, as specified by the DH parameters

#### Improved Output Text Formatting
Added more detail, visual cuing, etc.
because it's nice to be able to find

#### Added Printing of Date/Time of Run Start for
but could really have done better here by effective use of logging
Added more detail, visual cuing, etc.

#### Added forward kinematics for joint value verification (as recommended)

#### Modified script to run all test cases in a batch rather than manually specifying
would be nice to have the option to specify a test case, but it was more useful to me to get the whole rundown every time than to bother implementing that feature...

## Inverse Kinematics Steps
    1. Determine wrist center coordinates from target end effector position/orientation
    1. Calculate joint 1 angle (rotate arm's working plane to contain wc)
    1. Calculate cylindrical radius of wc from Z1
    1. 